<center>
<img src="http://www.bigdive.eu/wp-content/uploads/2012/05/logoBIGDIVE-01.png">
</center>

---

# Text Analysis

## André Panisson

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Loading the dataset
### The 20 Newsgroups data set

The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups

The data is organized into 20 different newsgroups, each corresponding to a different topic. Some of the newsgroups are very closely related to each other (e.g. comp.sys.ibm.pc.hardware / comp.sys.mac.hardware), while others are highly unrelated (e.g misc.forsale / soc.religion.christian).

In [15]:
from sklearn import datasets
dataset = datasets.fetch_20newsgroups()
documents = dataset.data

In [16]:
print documents[0]

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [4]:
len(documents)

11314

### Data cleaning

Removing from text pieces that do not convey any semantic meaning (e.g., mail headers, email adresses, host names...)

In [17]:
import re
from_re = re.compile(r"^From: .*\n")

print from_re.sub('', documents[0])

Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [18]:
for i in range(len(documents)):
    documents[i] = from_re.sub('', documents[i])
print documents[1]

Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



### Exercise: create a regular expression to remove the Nntp-Posting-Host header

In [19]:
nttp_re = re.compile(r"\nNNTP-Posting-Host: .*\n", flags=re.IGNORECASE) # nelle email NNTP e' scritto anche Nntp

for i in range(len(documents)):
    documents[i] = nttp_re.sub('\n', documents[i])
print documents[1]

Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [20]:
print documents[0]

Subject: WHAT car is this!?
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







# From Text Messages to Feature Vectors
We need to transform our text data into feature vectors, numerical representations which are suitable for performing statistical analysis. The most common way to do this is to apply a bag-of-words approach where the frequency of an occurrence of a word becomes a feature for our classifier.


## Term Frequency-Inverse Document Frequency

We want to consider the relative importance of particular words, so we'll use term frequency–inverse document frequency as a weighting factor. This will control for the fact that some words are more "spamy" than others.

## Mathematical details

tf–idf is the product of two statistics, term frequency and inverse document
frequency. Various ways for determining the exact values of both statistics
exist. In the case of the '''term frequency''' tf(''t'',''d''), the simplest
choice is to use the ''raw frequency'' of a term in a document, i.e. the
number of times that term ''t'' occurs in document ''d''. If we denote the raw
frequency of ''t'' by f(''t'',''d''), then the simple tf scheme is
tf(''t'',''d'') = f(''t'',''d''). Other possibilities
include:

  * boolean_data_type "frequencies": tf(''t'',''d'') = 1 if ''t'' occurs in ''d'' and 0 otherwise; 
  * logarithmically scaled frequency: tf(''t'',''d'') = log (f(''t'',''d'') + 1); 
  * augmented frequency, to prevent a bias towards longer documents, e.g. raw frequency divided by the maximum raw frequency of any term in the document: :$\mathrm{tf}(t,d) = 0.5 + \frac{0.5 \times \mathrm{f}(t, d)}{\max\{\mathrm{f}(w, d):w \in d\}}$

The '''inverse document frequency''' is a measure of whether the term is
common or rare across all documents. It is obtained by dividing the total
number of documents by the number of documents containing the
term, and then taking the logarithm of that quotient.

:$ \mathrm{idf}(t, D) = \log \frac{|D|}{|\{d \in D: t \in d\}|}$

with

  * $ |D| $: cardinality of D, or the total number of documents in the corpus 
  * $ |\{d \in D: t \in d\}| $ : number of documents where the term $ t $ appears (i.e., $ \mathrm{tf}(t,d) eq 0$). If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the formula to $1 + |\{d \in D: t \in d\}|$. 

Mathematically the base of the log function does not matter and constitutes a
constant multiplicative factor towards the overall result.

Then tf–idf is calculated as

$$\mathrm{tfidf}(t,d,D) = \mathrm{tf}(t,d) \times \mathrm{idf}(t, D)$$

In [21]:
from sklearn.feature_extraction import text

vectorizer = text.CountVectorizer(max_df=0.8, max_features=10000, stop_words=text.ENGLISH_STOP_WORDS)
counts = vectorizer.fit_transform(dataset.data)
tfidf = text.TfidfTransformer().fit_transform(counts)

In [23]:
counts

<11314x10000 sparse matrix of type '<type 'numpy.int64'>'
	with 942598 stored elements in Compressed Sparse Row format>

In [24]:
tfidf

<11314x10000 sparse matrix of type '<type 'numpy.float64'>'
	with 942598 stored elements in Compressed Sparse Row format>

In [26]:
counts[0].data

array([5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [27]:
tfidf[0].data

array([ 0.1930185 ,  0.1372247 ,  0.14510924,  0.08084941,  0.08820814,
        0.10135368,  0.10737348,  0.12239038,  0.15883363,  0.08893426,
        0.15909506,  0.14251793,  0.12974949,  0.06280941,  0.12787225,
        0.12000442,  0.14479169,  0.19483226,  0.13987913,  0.11040869,
        0.08313   ,  0.17022859,  0.10135368,  0.20354867,  0.12400092,
        0.13524559,  0.13256705,  0.15857461,  0.146923  ,  0.09737472,
        0.12415201,  0.19677119,  0.13030971,  0.09292594,  0.14559241,
        0.11455566,  0.16043998,  0.05440614,  0.55943454])

In [28]:
vectorizer.vocabulary_ # il value e' l'indice dove si trova nella matrice

{u'woods': 9800,
 u'hanging': 4318,
 u'broward': 1736,
 u'bringing': 1720,
 u'wednesday': 9692,
 u'shows': 8198,
 u'cfa': 2012,
 u'frederick': 3937,
 u'270': 307,
 u'272': 308,
 u'273': 309,
 u'274': 310,
 u'275': 311,
 u'278': 312,
 u'targa': 8818,
 u'errors': 3478,
 u'usenet': 9376,
 u'designing': 2948,
 u'kids': 5162,
 u'controversy': 2508,
 u'dna': 3125,
 u'inevitable': 4736,
 u'benedikt': 1500,
 u'intake': 4831,
 u'morally': 6001,
 u'wang': 9633,
 u'want': 9635,
 u'beyer': 1519,
 u'travel': 9119,
 u'wrong': 9845,
 u'fit': 3809,
 u'wiretapping': 9764,
 u'fix': 3812,
 u'fij': 3771,
 u'effects': 3325,
 u'sheep': 8159,
 u'6ql': 568,
 u'estimate': 3502,
 u'sys': 8774,
 u'needed': 6170,
 u'master': 5717,
 u'genesis': 4067,
 u'0d': 20,
 u'ahmet': 909,
 u'feeling': 3740,
 u'affairs': 874,
 u'vga': 9507,
 u'sy_': 8758,
 u'tech': 8847,
 u'saying': 7932,
 u'chelios': 2065,
 u'plate': 6860,
 u'altogether': 982,
 u'nicely': 6225,
 u'patch': 6673,
 u'widget': 9727,
 u'news': 6207,
 u'lots': 553

In [29]:
vectorizer.get_feature_names()

[u'00',
 u'000',
 u'005',
 u'01',
 u'02',
 u'02238',
 u'02p',
 u'03',
 u'030',
 u'0358',
 u'04',
 u'040',
 u'0400',
 u'05',
 u'06',
 u'07',
 u'08',
 u'09',
 u'0b',
 u'0c',
 u'0d',
 u'0el',
 u'0em',
 u'0g',
 u'0i',
 u'0l',
 u'0m',
 u'0p',
 u'0q',
 u'0qax',
 u'0t',
 u'0tbxn',
 u'0tq',
 u'0u',
 u'0w',
 u'10',
 u'100',
 u'1000',
 u'101',
 u'102',
 u'1024',
 u'1024x768',
 u'102nd',
 u'103',
 u'104',
 u'105',
 u'106',
 u'107',
 u'108',
 u'109',
 u'10k',
 u'10th',
 u'11',
 u'110',
 u'1100',
 u'111',
 u'112',
 u'113',
 u'114',
 u'115',
 u'116',
 u'117',
 u'118',
 u'119',
 u'12',
 u'120',
 u'1200',
 u'121',
 u'122',
 u'123',
 u'124',
 u'125',
 u'126',
 u'127',
 u'128',
 u'1280x1024',
 u'129',
 u'13',
 u'130',
 u'131',
 u'132',
 u'133',
 u'134',
 u'135',
 u'136',
 u'137',
 u'138',
 u'139',
 u'13p',
 u'13q',
 u'13qs',
 u'13s',
 u'14',
 u'140',
 u'1400',
 u'141',
 u'142',
 u'143',
 u'144',
 u'145',
 u'146',
 u'147',
 u'148',
 u'14853',
 u'149',
 u'15',
 u'150',
 u'1500',
 u'151',
 u'152',
 u'154',

In [30]:
counts[0].nonzero()[1]

array([1899, 9321, 5706, 2246, 6634,  105, 9796, 3433, 7930, 2791, 3161,
       8450, 5518, 5282, 3270,  583, 1850, 3162, 7425, 8306,  828, 1777,
       8083, 7642, 1620, 5200, 5958, 3421, 8421, 9938, 7107, 4449, 4759,
       5519, 5616, 8929, 4634, 1735, 6182], dtype=int32)

In [33]:
array(vectorizer.get_feature_names())[counts[0].nonzero()[1]]

array([u'car', u'university', u'maryland', u'college', u'park', u'15',
       u'wondering', u'enlighten', u'saw', u'day', u'door', u'sports',
       u'looked', u'late', u'early', u'70s', u'called', u'doors',
       u'really', u'small', u'addition', u'bumper', u'separate', u'rest',
       u'body', u'know', u'model', u'engine', u'specs', u'years',
       u'production', u'history', u'info', u'looking', u'mail', u'thanks',
       u'il', u'brought', u'neighborhood'], 
      dtype='<U80')

Se diamo il numero di ngrams tra 1 e 2 abbiamo anche le coppie di parole

In [34]:
from sklearn.feature_extraction import text

vectorizer = text.CountVectorizer(max_df=0.8, max_features=10000, stop_words=text.ENGLISH_STOP_WORDS, ngram_range=(1,2))
counts = vectorizer.fit_transform(dataset.data)
tfidf = text.TfidfTransformer().fit_transform(counts)

In [35]:
counts[0].data

array([5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
array(vectorizer.get_feature_names())[counts[0].nonzero()[1]]

array([u'car', u'university', u'maryland', u'college', u'park', u'15',
       u'wondering', u'saw', u'day', u'door', u'sports', u'looked',
       u'late', u'early', u'called', u'doors', u'really', u'small',
       u'addition', u'bumper', u'separate', u'rest', u'body', u'know',
       u'model', u'engine', u'specs', u'years', u'production', u'history',
       u'info', u'looking', u'mail', u'thanks', u'il', u'brought',
       u'neighborhood', u'organization university', u'university maryland',
       u'maryland college', u'college park', u'lines 15', u'mail thanks'], 
      dtype='<U80')

### Natural Language Processing with NLTK

NLTK (Natural Language ToolKit) is a library for symbolic and statistical natural language processing (NLP).

It supports a few functionalities for NLP, such as:

- Lexical analysis: Word and text tokenizer
- n-gram and collocations
- Part-of-speech tagger
- Tree model and Text chunker for capturing
- Named-entity recognition

In [39]:
import nltk
nltk.download('wordnet') # serve perche' manca questo pacchetto

[nltk_data] Downloading package wordnet to /home/bigdive/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [40]:
from nltk.corpus import wordnet as wn
from nltk import stem
import re

pattern = re.compile('(?u)\\b[A-Za-z]{3,}')

stemmer = stem.SnowballStemmer('english')
def stemming(doc):
    l = [stemmer.stem(t) for t in pattern.findall(doc)]
    return [w for w in l if len(w) > 2]

wnl = stem.WordNetLemmatizer()
def lemmatize(doc):
    
    def lemma(w):
        l = wnl.lemmatize(t, wn.NOUN)
        if l == w:
            l = wnl.lemmatize(t, wn.ADJ)
        if l == w:
            l = wnl.lemmatize(t, wn.ADV)
        if l == w:
            l = wnl.lemmatize(t, wn.VERB)
        return l
    
    l = [lemma(t) for t in pattern.findall(doc)]
    return [w for w in l if len(w) > 2]


In [41]:
sentence = """For grammatical reasons, documents are going to use different forms of a word,
such as organize, organizes, and organizing.
Additionally, there are families of derivationally related words with similar meanings,
such as democracy, democratic, and democratization."""

print sentence
print
print stemming(sentence)
print
print lemmatize(sentence)

For grammatical reasons, documents are going to use different forms of a word,
such as organize, organizes, and organizing.
Additionally, there are families of derivationally related words with similar meanings,
such as democracy, democratic, and democratization.

[u'for', u'grammat', u'reason', u'document', u'are', u'use', u'differ', u'form', u'word', u'such', u'organ', u'organ', u'and', u'organ', u'addit', u'there', u'are', u'famili', u'deriv', u'relat', u'word', u'with', u'similar', u'mean', u'such', u'democraci', u'democrat', u'and', u'democrat']

['For', 'grammatical', u'reason', u'document', 'use', 'different', u'form', 'word', 'such', 'organize', u'organize', 'and', u'organize', 'Additionally', 'there', u'family', 'derivationally', u'relate', u'word', 'with', 'similar', u'meaning', 'such', 'democracy', 'democratic', 'and', 'democratization']


In [42]:
vectorizer = text.CountVectorizer(max_df=0.95,
                                  max_features=10000,
                                  stop_words='english',
                                  encoding='latin1',
                                  tokenizer=lemmatize, # prende il testo e fa tornare una lista di parole
                                  ngram_range=(1, 2))
counts = vectorizer.fit_transform(dataset.data)
tfidf = text.TfidfTransformer().fit_transform(counts)

In [43]:
array(vectorizer.get_feature_names())[counts[0].nonzero()[1]]

array([u'car', u'university', u'maryland', u'college', u'park', u'wonder',
       u'enlighten', u'saw', u'day', u'door', u'sport', u'look', u'late',
       u'early', u'really', u'small', u'addition', u'bumper', u'separate',
       u'rest', u'body', u'know', u'model', u'engine', u'spec', u'year',
       u'production', u'make', u'history', u'info', u'mail', u'thank',
       u'bring', u'neighborhood', u'subject car',
       u'organization university', u'university maryland',
       u'maryland college', u'college park', u'park line', u'line wonder',
       u'sport car', u'mail thank'], 
      dtype='<U26')

Here, we create a variable, tfidf, which is a vectorizer responsible for performing three important steps:

- First, it will build a dictionary of features where keys are terms and values are indices of the term in the feature matrix (that's the fit part in fit_transform)
- Second, it will transform our documents into numerical feature vectors according to the frequency of words appearing in each text message. Since any one text message is short, each feature vector will be made up of mostly zeros, each of which indicates that a given word appeared zero times in that message.
- Lastly, it will compute the tf-idf weights for our term frequency matrix.

## Nonnegative Matrix Factorization for Topic extraction



Imagine having 5 documents, 2 of them about environment and 2 of them about U.S. Congress and 1 about both, that means it says about government legislation process in protecting an environment. We need to write a program that unmistakably identifies category of each document and also returns a degree of belonging of each document to a particular category. For this elementary example we limit our vocabulary to 5 words: AIR, WATER, POLLUTION, DEMOCRAT, REPUBLICAN. Category ENVIRONMENT and category CONGRESS may contain all 5 words but with different probability. We understand that the word POLLUTION has more chances to be in the article about ENVIRONMENT than in the article about CONGRESS, but can theoretically be in both. Presume after an examination of our data we built following document-term table:

<table border="" cellpadding="3" style="font-family:'Times New Roman'">
<tbody>
<tr>
<td>document/word</td>
<td>air</td>
<td>water</td>
<td>pollution</td>
<td>democrat</td>
<td>republican</td>
</tr>
<tr>
<td>doc 1</td>
<td>3</td>
<td>2</td>
<td>8</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>doc 2</td>
<td>1</td>
<td>4</td>
<td>12</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>doc 3</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>10</td>
<td>11</td>
</tr>
<tr>
<td>doc 4</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>8</td>
<td>5</td>
</tr>
<tr>
<td>doc 5</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
</tbody>
</table>

We distinguish our categories by the group of words assigned to them. We decide that category ENVIRONMENT normally should contain only words AIR, WATER, POLLUTION and category CONGRESS should contain only words DEMOCRAT and REPUBLICAN. We build another matrix, each row of which represent category and contains counts for only words that assigned to each category. 

<table border="" cellpadding="3" style="font-family:'Times New Roman'">
<tbody>
<tr>
<td>categories</td>
<td>air</td>
<td>water</td>
<td>pollution</td>
<td>democrat</td>
<td>republican</td>
</tr>
<tr>
<td>ENVIRONMENT</td>
<td>5</td>
<td>7</td>
<td>21</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>CONGRESS</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>19</td>
<td>17</td>
</tr>
</tbody>
</table>

We change values from frequencies to probabilities by dividing them by sums in rows, which turns each row into probability distribution.

<table border="" cellpadding="3" style="font-family:'Times New Roman'">
<caption>Matrix&nbsp;<strong>H</strong></caption>
<tbody>
<tr>
<td>categories</td>
<td>air</td>
<td>water</td>
<td>pollution</td>
<td>democrat</td>
<td>republican</td>
</tr>
<tr>
<td>ENVIRONMENT</td>
<td>0.15</td>
<td>0.21</td>
<td>0.64</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>CONGRESS</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>0.53</td>
<td>0.47</td>
</tr>
</tbody>
</table>

Now we create another matrix that contains probability distribution for categories within each document that looks like follows:

<table border="" cellpadding="3" style="font-family:'Times New Roman'">
<caption>Matrix&nbsp;<strong>W</strong></caption>
<tbody>
<tr>
<td>documents</td>
<td>ENVIRONMENT</td>
<td>CONGRESS</td>
</tr>
<tr>
<td>doc 1</td>
<td>1.0</td>
<td>0.0</td>
</tr>
<tr>
<td>doc 2</td>
<td>1.0</td>
<td>0.0</td>
</tr>
<tr>
<td>doc 3</td>
<td>0.0</td>
<td>1.0</td>
</tr>
<tr>
<td>doc 4</td>
<td>0.0</td>
<td>1.0</td>
</tr>
<tr>
<td>doc 5</td>
<td>0.6</td>
<td>0.4</td>
</tr>
</tbody>
</table>

It shows that top two documents speak about environment, next two about congress and last document about both. Ratios 0.6 and 0.4 for the last document are defined by 3 words from environment category and 2 words from congress category. Now we multiply both matrices and compare the result with original data but in a normalized form. Normalization in this case is division of each row by the sum of all elements in rows. The comparison is shown side-by-side below:

<table cellpadding="10" style="font-family:'Times New Roman'">
<tbody>
<tr>
<td>
<table border="" cellpadding="3">
<caption>Product of&nbsp;<strong>W * H</strong></caption>
<tbody>
<tr>
<td>0.15</td>
<td>0.21</td>
<td>0.64</td>
<td>0.0</td>
<td>0.0</td>
</tr>
<tr>
<td>0.15</td>
<td>0.21</td>
<td>0.64</td>
<td>0.0</td>
<td>0.0</td>
</tr>
<tr>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.53</td>
<td>0.47</td>
</tr>
<tr>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.53</td>
<td>0.47</td>
</tr>
<tr>
<td>0.09</td>
<td>0.13</td>
<td>0.38</td>
<td>0.21</td>
<td>0.19</td>
</tr>
</tbody>
</table>
</td>
<td>
<table border="" cellpadding="3">
<caption>Normalized data&nbsp;<strong>N</strong></caption>
<tbody>
<tr>
<td>0.23</td>
<td>0.15</td>
<td>0.62</td>
<td>0.0</td>
<td>0.0</td>
</tr>
<tr>
<td>0.06</td>
<td>0.24</td>
<td>0.70</td>
<td>0.0</td>
<td>0.0</td>
</tr>
<tr>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.48</td>
<td>0.52</td>
</tr>
<tr>
<td>0.0</td>
<td>0.0</td>
<td>0.0</td>
<td>0.61</td>
<td>0.39</td>
</tr>
<tr>
<td>0.2</td>
<td>0.2</td>
<td>0.2</td>
<td>0.2</td>
<td>0.2</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>

The correlation is obvious. The problem definition is to find constrained matrices W and H (given the number of categories), product of which is the best match with normalized data N. When approximation is found matrix H will contain sought categories.

**Formally**, we are trying to minimize this:

$$ \|\mathbf{N} - \mathbf{WH}\|^2_F $$

In [44]:
from sklearn import decomposition

# Fit the NMF model
nmf = decomposition.NMF(n_components=6)
nmf.fit(tfidf)
W = nmf.transform(tfidf)
H = nmf.components_

In [45]:
W.shape

(11314, 6)

In [46]:
H.shape

(6, 10000)

In [53]:
H[0].argsort()[:-21:-1]

array([2721, 1600, 9825,  480, 6422, 1221, 3757, 7610, 2542, 8781, 4598,
       4951, 7385, 5212, 3617, 8263, 8849, 4984, 9930, 4724])

In [54]:
# Inverse the vectorizer vocabulary to be able
feature_names = vectorizer.get_feature_names()

In [56]:
print [feature_names[i] for i in H[0].argsort()[:-21:-1]]

[u'edu', u'com', u'write', u'article', u'people', u'car', u'gun', u'say', u'don', u'think', u'just', u'like', u'right', u'make', u'good', u'state', u'time', u'line article', u'year', u'know']


In [57]:
for topic_idx, topic in enumerate(H):
    print "Topic #%d:" % topic_idx
    print ",".join([feature_names[i] for i in topic.argsort()[:-21:-1]])
    print

Topic #0:
edu,com,write,article,people,car,gun,say,don,think,just,like,right,make,good,state,time,line article,year,know

Topic #1:
window,use,file,card,drive,thank,program,problem,driver,university,disk,edu,run,help,mail,scsi,work,graphic,version,software

Topic #2:
god,christian,jesus,bible,say,believe,people,faith,christ,religion,atheist,belief,church,life,think,know,truth,christianity,sin,mean

Topic #3:
pitt,pitt edu,gordon bank,geb,gordon,bank,edu,geb pitt,edu gordon,cadre,bank skepticism,chastity intellect,skepticism chastity,cadre dsl,dsl pitt,dsl,intellect geb,geb cadre,surrender soon,shameful surrender

Topic #4:
game,team,player,play,win,year,hockey,season,score,edu,baseball,nhl,fan,league,good,playoff,university,pitch,toronto,think

Topic #5:
key,chip,clipper,encryption,escrow,clipper chip,government,use,algorithm,phone,crypto,security,key escrow,secret,nsa,encrypt,secure,public,wiretap,privacy



# Exercise

Compare the results of Nonnegative Matrix Factorization (NMF) with Latent Dirichlet Allocation (LDA).

In [59]:
from sklearn import decomposition

# Fit the NMF model
nmf = decomposition.LatentDirichletAllocation(n_topics=6)
nmf.fit(tfidf)
W = nmf.transform(tfidf)
H = nmf.components_

In [60]:
# Inverse the vectorizer vocabulary to be able
feature_names = vectorizer.get_feature_names()

In [61]:
print [feature_names[i] for i in H[0].argsort()[:-21:-1]]

[u'optilink', u'cramer', u'gld', u'expo lcs', u'expo', u'columbia', u'columbia edu', u'xpert', u'clayton', u'lcs mit', u'lcs', u'internet line', u'organization internet', u'xpert expo', u'uoknor edu', u'uoknor', u'gay', u'homosexual', u'callison', u'optilink com']


In [62]:
for topic_idx, topic in enumerate(H):
    print "Topic #%d:" % topic_idx
    print ",".join([feature_names[i] for i in topic.argsort()[:-21:-1]])
    print

Topic #0:
optilink,cramer,gld,expo lcs,expo,columbia,columbia edu,xpert,clayton,lcs mit,lcs,internet line,organization internet,xpert expo,uoknor edu,uoknor,gay,homosexual,callison,optilink com

Topic #1:
edu,write,people,say,god,article,think,com,don,make,year,know,good,just,university,time,like,game,right,line article

Topic #2:
key,chip,scsi,clipper,encryption,armenian,ide,netcom,simms,cleveland,escrow,clipper chip,turkish,communication,case western,western reserve,reserve university,organization case,nsa,bus

Topic #3:
pitt,urbana,gordon bank,geb,gordon,bank,pitt edu,illinois urbana,urbana line,psu,psu edu,udel,udel edu,psuvm,psuvm psu,monash,geb pitt,edu gordon,athens,georgia

Topic #4:
stratus,sandvik,kent,stratus com,apple com,uicvm,uic,uic edu,uicvm uic,cdt,apple,sandvik newton,newton apple,newton,kent sandvik,singapore,atf,ranch,subject atf,stratus computer

Topic #5:
use,com,window,edu,university,thank,drive,write,know,file,like,need,work,problem,article,distribution,card,mai

I risultati del LDA sono piu' criptici.